In [1]:
import xarray as xr 
import numpy as np

from dask import delayed
from tqdm import tqdm
from dask.diagnostics import ProgressBar
import dask
from dask import delayed, compute
import glob
from dask.distributed import Client

import zarr
from numcodecs import Blosc

import gc

import sys, os 
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))


from wofscast.data_generator import (load_wofscast_data, 
                                    wofscast_data_generator, 
                                    wofscast_batch_generator, 
                                    to_static_vars,
                                    add_local_solar_time
                                    
                                    )
from wofscast.wofscast_task_config import WOFS_TASK_CONFIG, train_lead_times, TARGET_VARS
from wofscast import data_utils
import dataclasses

from dask.distributed import performance_report


In [2]:
def read_mfnetcdfs_dask(paths, dim, transform_func=None, load=True):
    """Read multiple NetCDF files into memory, using Dask for parallel loading."""
    # Absolutely, crucial to set threads_per_worker=1!!!!
    # https://forum.access-hive.org.au/t/netcdf-not-a-valid-id-errors/389/19
    #To summarise in this thread, it looks like a work-around in netcdf4-python to deal 
    #with netcdf-c not being thread safe was removed in 1.6.1. 
    #The solution (for now) is to make sure your cluster only uses 1 thread per worker.

    dataset = xr.open_mfdataset(paths, concat_dim=dim, combine='nested',
                                parallel=True, preprocess=transform_func,
                                chunks={"time": 4}, )  # Adjust the chunking strategy as needed
    if load:
        with ProgressBar():
            loaded_dataset= dataset.compute()
        return loaded_dataset

    return dataset 

In [3]:
n_files = 8192
idx = 128
data_paths = glob.glob('/work/mflora/wofs-cast-data/datasets/2019/*.nc')

In [4]:
client = Client(n_workers=16, threads_per_worker=1) 
client     

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 503.42 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38253,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 503.42 GiB
Comm: tcp://127.0.0.1:33013,Total threads: 1
Dashboard: http://127.0.0.1:36185/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:35547,


2024-05-09 10:47:55,450 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:35047'.
2024-05-09 10:48:09,362 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,366 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,370 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,374 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,379 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,383 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,386 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,389 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,392 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,396 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,400 - distributed.nanny - WARNING - Restarting worker
2024-05-09 10:48:09,403 - distributed.nanny - WARNING - R

In [5]:
%%time

# 8192 files -> GBs



# 4096 files -> 69GBs
# CPU times: user 3min 21s, sys: 28.1 s, total: 3min 49s
# Wall time: 16min 15s

# 1024 files -> 17GB
# CPU times: user 39.3 s, sys: 5.07 s, total: 44.4 s
# Wall time: 2min 24s 

# Using 12 workers, 
# Using 16 workers, 1min 38s
# Using 32 workers, 1min 23s 
# Using 64 workers, no bueno 

def read_mfnetcdfs_dask(paths, dim, transform_func=None, load=True):
    """Read multiple NetCDF files into memory, using Dask for parallel loading."""
    # Absolutely, crucial to set threads_per_worker=1!!!!
    # https://forum.access-hive.org.au/t/netcdf-not-a-valid-id-errors/389/19
    #To summarise in this thread, it looks like a work-around in netcdf4-python to deal 
    #with netcdf-c not being thread safe was removed in 1.6.1. 
    #The solution (for now) is to make sure your cluster only uses 1 thread per worker.

    dataset = xr.open_mfdataset(paths, concat_dim=dim, combine='nested',
                                parallel=True, preprocess=transform_func,
                                chunks={"time": 4}, )  # Adjust the chunking strategy as needed
    if load:
        with ProgressBar():
            loaded_dataset= dataset.compute()
        return loaded_dataset

    return dataset 

n_files = 8192
idx = 128
data_paths = glob.glob('/work/mflora/wofs-cast-data/datasets/2019/*.nc')

files = data_paths[idx:idx + n_files]
dataset = read_mfnetcdfs_dask(files, dim='batch', transform_func=add_local_solar_time, load=False) 
    
# Initialize Blosc compressor with LZ4 for speed
compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.BITSHUFFLE)
    
dataset.to_zarr(f'/work/mflora/wofs-cast-data/wofcast_dataset_n_samples-{n_files}.zarr', 
                    mode='w', 
                   consolidated=True, 
                   encoding={var: {'compressor': compressor} for var in dataset.variables})

/work/mflora/miniconda3/envs/wofs-cast/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 63.38 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-05-09 10:47:45,261 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('broadcast_to-concatenate-concatenate-store-map-e68488012184d735861e7b07e042dd5c', 6566, 0, 0, 0, 0))" coro=<Worker.execute() done, defined at /work/mflora/miniconda3/envs/wofs-cast/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2024-05-09 10:47:45,261 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('broadcast_to-concatenate-concatenate-store-map-e68488012184d735861e7b07e042dd5c', 657, 0, 0, 0, 0))" coro=<Worker.execute() done, defined at /work/mflora/miniconda3/envs/wofs-cast/lib/python3.10/site-packages/distribut

2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-05-09 10:47:47,815 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Da

2024-05-09 10:47:55,451 - distributed.worker - ERROR - Scheduler was unaware of this worker 'tcp://127.0.0.1:35047'. Shutting down.


KeyboardInterrupt: 

In [8]:
client.close()

In [7]:
def wofscast_data_generator(path='/work/mflora/wofs-cast-data/wofcast_dataset.zarr', 
                            train_lead_times=train_lead_times,
                            batch_chunk_size=256, 
                    client=None, 
                    task_config=None):

    with xr.open_dataset(path, 
                         engine='zarr', 
                         consolidated=True, 
                         chunks={'batch' : batch_chunk_size}
                        ) as ds:
        
        total_samples = len(ds.batch)
        total_batches = total_samples // batch_chunk_size + (1 if total_samples % batch_chunk_size > 0 else 0)
    
        for batch_num in tqdm(range(total_batches), desc='Loading Data..'):
            start_idx = batch_num * batch_chunk_size
            end_idx = min((batch_num + 1) * batch_chunk_size, total_samples)
            batch_indices = slice(start_idx, end_idx)  # Use slice for more efficient indexing
        
            print(batch_num, batch_indices) 
        
            # Load this batch into memory. 
            this_batch = ds.isel(batch=batch_indices)
        
            inputs, targets, forcings = data_utils.extract_inputs_targets_forcings(
                this_batch,
                target_lead_times=train_lead_times,
                **dataclasses.asdict(task_config)
            )
        
            inputs = to_static_vars(inputs)
        
            inputs = inputs.transpose('batch', 'time', 'lat', 'lon', 'level')
            targets = targets.transpose('batch', 'time', 'lat', 'lon', 'level')
            forcings = forcings.transpose('batch', 'time', 'lat', 'lon')

            inputs, targets, forcings = dask.compute(inputs, targets, forcings)
            
            yield inputs, targets, forcings